# Azure AI Foundry Agent Service - Single Agent MCP

Azure AI FoundryのMCP（Model Context Protocol）ツールを使用して外部サービスと連携するエージェントを構築するノートブックです。

## 概要

このノートブックでは、MCPツールを活用してMicrosoft Learn公式ドキュメントを検索できるエージェントを作成します。MCPプロトコルにより、エージェントが外部のAPIやサービスと安全かつ標準化された方法で連携する方法を学習します。

### 主な機能
- **MCP Tool**: Model Context Protocolによる外部サービス連携
- **ツール承認機能**: 自動承認と手動承認の両方をサポート

### MCPの利点
- **標準化**: 統一されたプロトコルによる外部連携
- **セキュリティ**: 承認プロセスや Entra ID 認証による安全な外部アクセス
- **拡張性**: 様々な外部サービスとの連携が可能
- **透明性**: ツール実行の詳細な記録と確認

このエージェントは、企業の内部システム連携、外部API統合、サードパーティサービス連携などの実用的なアプリケーションに応用できます。

# ライブラリのインポート

必要なPythonライブラリとAzure AI Foundry SDKをインポートします。

In [37]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
)

# 環境変数の取得

Azure AI Foundryへの接続に必要な設定情報を環境変数から取得します。

- `PROJECT_ENDPOINT`: Azure AI Foundryプロジェクトのエンドポイント
- `AZURE_DEPLOYMENT_NAME`: 使用するAIモデルのデプロイメント名

これらの値は事前に.envファイルに設定されている必要があります。

In [38]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

Azure AI Foundryプロジェクトへの接続とエージェント管理用のクライアントを初期化します。

- `AIProjectClient`: Azure AI Foundryプロジェクトとの接続を管理
- `DefaultAzureCredential`: Azure認証（Azure CLI、Managed Identity等を自動検出）
- `agents_client`: エージェントの作成・管理・実行を行うクライアント

In [39]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

エージェントの実行結果を表示するためのヘルパー関数を定義します。

`agent_run_outputs`関数は以下の機能を提供します：
- スレッド内のメッセージ一覧を取得・表示
- 画像コンテンツがある場合は保存・表示
- ツール呼び出し情報の詳細表示（Run Stepsから取得）

In [40]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images", show_tool_calls=True, run_id=None):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像・ツール呼び出し）をNotebook上に表示＆画像は保存。
    
    Args:
        thread_id: スレッドID
        agents_client: エージェントクライアント
        target_dir: 画像保存ディレクトリ
        show_tool_calls: ツール呼び出し情報を表示するかどうか
        run_id: 特定のRunのツール呼び出し情報を表示する場合のRun ID
    """
    from IPython.display import Image, display
    from azure.ai.agents.models import RunStepToolCallDetails, RunStepFunctionToolCall, RunStepMessageCreationDetails
    
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    # メッセージの重複防止
    displayed_message_ids = set()
    
    # メッセージの表示
    for message in messages:
        # メッセージの重複チェック
        if message.id in displayed_message_ids:
            continue
        displayed_message_ids.add(message.id)
        
        print(f"\n{'='*60}")
        print(f"MESSAGE ROLE: {message.role.upper()}")
        print(f"MESSAGE ID: {message.id}")
        print(f"{'='*60}")
        
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents") and message.image_contents:
            print(f"\n[IMAGES]")
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"  Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))
    
    # ツール呼び出し情報の表示（Run Stepsから取得）
    if show_tool_calls and run_id:
        print(f"\n{'='*60}")
        print(f"RUN STEPS INFORMATION (RUN ID: {run_id})")
        print(f"{'='*60}")
        
        try:
            # Run Stepsを取得（デフォルトは新しい順なので、古い順に並び替え）
            run_steps = agents_client.run_steps.list(thread_id=thread_id, run_id=run_id)
            run_steps_list = list(run_steps)
            run_steps_list.reverse()  # 実行順序に並び替え（STEP1から順番に）
            
            # 重複防止のためのセット
            displayed_step_ids = set()
            
            print(f"Total Run Steps: {len(run_steps_list)}")
            
            # 全てのrun stepsを実行順序で表示
            for step_num, run_step in enumerate(run_steps_list, 1):
                # 重複チェック
                if run_step.id in displayed_step_ids:
                    print(f"[STEP {step_num}] - SKIPPED (Duplicate Step ID: {run_step.id})")
                    continue
                displayed_step_ids.add(run_step.id)
                
                print(f"\n[STEP {step_num}] - {run_step.type}")
                print(f"  Step ID: {run_step.id}")
                print(f"  Status: {run_step.status}")
                
                # Message Creation Step
                if isinstance(run_step.step_details, RunStepMessageCreationDetails):
                    print(f"  Message Creation Step")
                    if hasattr(run_step.step_details.message_creation, 'message_id'):
                        print(f"  Message ID: {run_step.step_details.message_creation.message_id}")
                
                # Tool Calls Step
                elif isinstance(run_step.step_details, RunStepToolCallDetails):
                    print(f"  Tool Calls Step - {len(run_step.step_details.tool_calls)} tool(s)")
                    
                    for tool_num, tool_call in enumerate(run_step.step_details.tool_calls, 1):
                        print(f"\n    [TOOL CALL {tool_num}]")
                        print(f"    Tool Type: {tool_call.type}")
                        print(f"    Tool Call ID: {tool_call.id}")
                        
                        # Function Tool Call の詳細
                        if isinstance(tool_call, RunStepFunctionToolCall):
                            print(f"    Function Name: {tool_call.function.name}")
                            print(f"    Function Arguments: {tool_call.function.arguments}")
                            # 関数の実行結果を表示（利用可能な場合）
                            if hasattr(tool_call.function, 'output') and tool_call.function.output:
                                print(f"    Function Output: {tool_call.function.output}")
                            elif hasattr(tool_call.function, 'outputs') and tool_call.function.outputs:
                                print(f"    Function Outputs: {tool_call.function.outputs}")
                            elif hasattr(tool_call.function, 'result') and tool_call.function.result:
                                print(f"    Function Result: {tool_call.function.result}")
                        
                        print(f"    {'-'*30}")
                
                # その他のステップタイプ
                else:
                    print(f"  Step Type: {type(run_step.step_details).__name__}")
                
                print(f"  Created At: {run_step.created_at}")
                if hasattr(run_step, 'completed_at') and run_step.completed_at:
                    print(f"  Completed At: {run_step.completed_at}")
                
                print(f"  {'='*50}")
                
        except Exception as e:
            print(f"Error retrieving run steps: {e}")
            print(f"Run ID: {run_id}, Thread ID: {thread_id}")

# ツールの定義

MCP（Model Context Protocol）ツールを設定し、外部サービスとの連携を準備します。

## MCPツールとは
Model Context Protocolは、言語モデルが外部ツール、メモリ、コンテキストと安全で構造化された状態を保持する方法で相互作用するための公開プロトコルです。

### MCPのアーキテクチャ
1. **Hosts**: MCPクライアントを使用してMCPサーバーからデータを接続・消費するアプリケーション
2. **Clients**: MCPホストのコンポーネントで、接続を管理しMCPサーバーからデータを取得
3. **Servers**: データリソース、アクション実行ツール、相互作用を導くプロンプトを提供するプログラム

### MCPツール設定項目
- **server_label**: MCPサーバーの一意識別子
- **server_url**: APIのエンドポイント
- **allowed_tools**: 利用可能なツールのリスト（オプション）
- **headers**: 認証やその他の目的でMCPサーバーに送信するHTTPヘッダー（オプション）

### 承認モード（require_approval）の詳細設定
1. **always**: すべてのツール呼び出しで承認が必要（デフォルト）
2. **never**: 承認不要
3. **{"never":["tool_name1", "tool_name2"]}**: 指定されたツールは承認不要
4. **{"always":["tool_name1", "tool_name2"]}**: 指定されたツールは承認必要

### セキュリティと認証
- **Entra ID統合**: Azure Identity ライブラリを通じたベストプラクティス認証
- **透明性**: 外部MCPサーバーとの共有データの明示的な承認プロセス
- **監査ログ**: リモートMCPサーバーとの共有データのロギング推奨

In [41]:
mcp_tool = McpTool(
    server_label="MicrosoftDocs",
    server_url="https://learn.microsoft.com/api/mcp",
)

# MCP ツール実行時の承認モードを never に設定
mcp_tool.set_approval_mode("never")

print(mcp_tool.resources)

{'mcp': [{'server_label': 'MicrosoftDocs', 'headers': {}, 'require_approval': 'never'}]}


# エージェントの作成

Foundry エージェントを作成します。

In [42]:
mcp_agent = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="mcp_agent",
    instructions=(
        "あなたは、MCPツールを使用してユーザーを支援できる有用なエージェントです。"
        "利用可能なMCPツールを使用して、質問に答えてタスクを実行します。"    
    ),
    description=(
        "あなたは Microsoft の公式ドキュメントを検索できる MCP ツールを持っているエージェントです。"
    ),
    tools=mcp_tool.definitions,
    tool_resources=mcp_tool.resources,
)
print(f"Created Agent. AGENT_ID: {mcp_agent.id}")


Created Agent. AGENT_ID: asst_cESwJdjt53tGWtixN0fdB8Fi


In [43]:
agent_dict = mcp_agent.as_dict()
print(json.dumps(agent_dict, indent=2, ensure_ascii=False))

{
  "id": "asst_cESwJdjt53tGWtixN0fdB8Fi",
  "object": "assistant",
  "created_at": 1753813667,
  "name": "mcp_agent",
  "description": "あなたは Microsoft の公式ドキュメントを検索できる MCP ツールを持っているエージェントです。",
  "model": "gpt-4.1",
  "instructions": "あなたは、MCPツールを使用してユーザーを支援できる有用なエージェントです。利用可能なMCPツールを使用して、質問に答えてタスクを実行します。",
  "tools": [
    {
      "type": "mcp",
      "server_label": "MicrosoftDocs",
      "server_url": "https://learn.microsoft.com/api/mcp",
      "allowed_tools": null
    }
  ],
  "top_p": 1.0,
  "temperature": 1.0,
  "tool_resources": {},
  "metadata": {},
  "response_format": "auto"
}


# スレッドの作成

エージェントとの対話を管理するスレッドを作成します。

In [44]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_eOO5Dmi4AaMyzQh7X9VFc8vf


# ユーザーメッセージの追加

スレッドにユーザーからの質問やリクエストを追加します。

In [45]:
# メッセージの追加
user_message = "Azure AI Foundry Agent Service の最新情報を教えてください。"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_24PTlnoLWhMOULU77TYKdO6r


# Run の実行

エージェントにタスクを実行させ、結果を取得します。

## MCP ツール自動承認で実行

承認モードを"never"に設定したMCPツールを使用して、自動承認でエージェントを実行します。

In [46]:
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=mcp_agent.id,
    tool_resources=mcp_tool.resources # MCP ツールのリソース情報を渡す（承認モード設定やリクエストヘッダーの設定はここで定義される）
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, agents_client, show_tool_calls=True, run_id=run.id)


MESSAGE ROLE: USER
MESSAGE ID: msg_24PTlnoLWhMOULU77TYKdO6r
Azure AI Foundry Agent Service の最新情報を教えてください。

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_ypmq89HTmgtY2EnGrwqiMowT
Azure AI Foundry Agent Serviceの最新情報は、以下の通りです（2025年6月〜7月時点の公式情報にもとづきます）。

### 主な新機能・アップデート（2025年6月以降）

- **Deep Researchツール**（2025年6月）
  - Azure OpenAIのo3-deep-researchモデルとBing Searchを用いた、高度な多段階リサーチ機能をエージェントに追加。
  - 詳細：[Deep Research tool](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research)

- **Model Context Protocol (MCP) ツール**（2025年6月）
  - エージェントをMCP（Model Context Protocol）でホストされるリモートツールと接続できるように拡張。
  - 詳細：[MCPツールについて](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol)

- **一般提供開始（GA）**（2025年5月）
  - サービスがGA（General Availability、正式リリース）となり、多数の新機能が追加。
    - Visual Studio Code拡張機能が利用可能に
    - Connected Agents（マルチエージェント連携機能）、Traceエージェント（詳細トレース&デバッグ）、Logic Apps連携（自動イベントトリガー）、Bing Custom SearchやMorningstar Toolなど新ツール追加
  - 詳細：[What's new in Az

## MCP ツール手動承認で実行

MCPツールの実行に対して手動で承認を行うプロセスを実装します。

### 手動承認の実装流れ
1. **Run作成**: `create()`でRunを作成（`create_and_process()`は使用しない）
2. **状態監視**: `requires_action`状態になるまでポーリング
3. **承認判断**: `SubmitToolApprovalAction`が要求されたら承認処理
4. **ToolApproval作成**: 各ツール呼び出しに対して承認オブジェクトを作成
5. **承認送信**: `submit_tool_outputs()`で承認結果を送信

### 手動承認の利点
- **セキュリティ**: 外部API呼び出しの詳細確認と制御
- **透明性**: ツール実行内容の事前確認
- **コスト管理**: 不要な外部API呼び出しの防止
- **コンプライアンス**: 企業ポリシーに応じた承認プロセス

### 承認オブジェクトの設定
- **tool_call_id**: 承認対象のツール呼び出しID
- **approve**: True（承認）またはFalse（拒否）
- **headers**: MCPツールのヘッダー情報

In [47]:
# Thread の作成
thread_1 = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread_1.id}")

# メッセージの追加
user_message = "Azure AI Foundry Agent Service の最新情報を教えてください。"
message = agents_client.messages.create(
    thread_id=thread_1.id,
    role="user",
    content=user_message,
)
print(f"Added Message. MESSAGE_ID: {message.id}")

# Run の作成と実行
run_1 = agents_client.runs.create(
    thread_id=thread_1.id,
    agent_id=mcp_agent.id
)

# Run の状態をポーリングして、必要なアクションがあるか確認
while True:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread_1.id, run_id=run_1.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread_1.id, run_id=run_1.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,  # 承認する場合は True、拒否する場合は False
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            # ツール承認を送信
            agents_client.runs.submit_tool_outputs(
                thread_id=thread_1.id,
                run_id=run_1.id,
                tool_approvals=tool_approvals
            )

    print(f"Current run status: {run.status}")
    if run.status not in ["queued", "in_progress", "requires_action"]:
        break


Created Thread. THREAD_ID: thread_GcdRrqwlQEVWQ6MEpsuJQh5K
Added Message. MESSAGE_ID: msg_YoRvrM4RG00bcZ0AcztXaakp
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Approving tool call: {'id': 'call_SGSdFy6yNERv1PPicqmeJYsA', 'type': 'mcp', 'arguments': '{"question":"Azure AI Foundry Agent Service ã\x81®æ\x9c\x80æ\x96°æ\x83\x85å\xa0±"}', 'name': 'microsoft_docs_search', 'server_label': 'MicrosoftDocs'}
tool_approvals: [{'tool_call_id': 'call_SGSdFy6yNERv1PPicqmeJYsA', 'approve': True, 'headers': {}}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PRO

In [48]:
agent_run_outputs(thread_1.id, agents_client, show_tool_calls=True, run_id=run_1.id)


MESSAGE ROLE: USER
MESSAGE ID: msg_YoRvrM4RG00bcZ0AcztXaakp
Azure AI Foundry Agent Service の最新情報を教えてください。

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_O1Fjv9d7OGBNjD8SVZHnp6An
Azure AI Foundry Agent Serviceの最新情報は、2025年6月時点で以下の主なアップデートがあります。

---

### 2025年6月

- **Deep Research tool**  
  Azure OpenAIのo3-deep-researchモデルとBing Searchを知識ソースにしたマルチステップリサーチプロセスが利用可能に。[詳細はこちら](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research)

- **Model Context Protocol (MCP) tool**  
  エージェントの機能拡張として、リモートのMCPサーバー上のツールと接続できるようになりました。[詳細はこちら](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol)

---

### 2025年5月

- **一般提供 (GA)**  
  Azure AI Foundry Agent Serviceが一般提供(GA)となりました。
    - Visual Studio Code拡張でエージェントのデプロイ・構成が可能
    - **Connected agents**：複数エージェントの連携によるマルチエージェント開発
    - **Trace agents**：各エージェントの入力・出力・実行順を詳細にトレース
    - Azure Logic Appsによるトリガー自動化
    - 新しいツール：Bing Custom Search/Morningstar、他

---

### 2025年4月・3月の主な機能拡張

- Azure

# トレースの確認

[Azure AI Foundry Portal](https://ai.azure.com/?cid=learnDocs) でエージェントの実行トレースを確認してみましょう。

# Agent ID を .env ファイルに保存

作成したFile SearchエージェントのIDを永続化し、他のノートブックで再利用できるようにします。

In [49]:
# 変数の定義
agent_env_key = "FOUNDRY_MCP_AGENT_ID"
agent_env_value = mcp_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')


.envファイルに FOUNDRY_MCP_AGENT_ID="asst_cESwJdjt53tGWtixN0fdB8Fi" を追記しました。
